In [1]:
import findspark 
findspark.init()
import py4j

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('practice2').getOrCreate()

In [4]:
spark

In [5]:
lines = spark.sparkContext.textFile('/home/vkkr125/programming/bigdata/u.data')

In [6]:
lines.collect()

['196 211 3 883848348',
 '323 32 3 848948924',
 '311 22 1 824839853',
 '22 3777 2 84839589',
 '111 844 1 84394389']

In [7]:
df_pyspark = spark.read.csv('/home/vkkr125/Downloads/yellow_tripdata_2018-01.csv',header = True,inferSchema = True)
# df_pyspark.show(10)
df_pyspark.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [8]:

# number of columns
df_pyspark.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount']

In [9]:
# number of rows in dataframe

df_pyspark.count()

8759874

In [10]:
df_pyspark.select(df_pyspark['total_amount'] >=10).show(10)

+--------------------+
|(total_amount >= 10)|
+--------------------+
|               false|
|                true|
|               false|
|                true|
|                true|
|               false|
|                true|
|               false|
|               false|
|               false|
+--------------------+
only showing top 10 rows



In [11]:

df_pyspark.na.drop().count()

#count of data after droping the null values , same as previous 
# so the data is cleaned there is no null values in dataframe 

8759874

###  Task 1

In [13]:
# Query 1. - Add a column named as "Revenue" into dataframe which is the sum of the below columns 'Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount'

df_pyspark = df_pyspark.withColumn('Revenue',(df_pyspark['fare_amount'] + df_pyspark['extra'] + df_pyspark['mta_tax'] \
                      + df_pyspark['improvement_surcharge'] + df_pyspark['tip_amount'] + df_pyspark['tolls_amount'] \
                      + df_pyspark['total_amount']))
df_pyspark.select('Revenue').show(10)

+------------------+
|           Revenue|
+------------------+
|              11.6|
|              30.6|
|              16.6|
|              69.6|
|              33.1|
|              11.6|
|24.700000000000003|
|              12.6|
|              17.0|
|              13.6|
+------------------+
only showing top 10 rows



In [91]:
# number of columns now
df_pyspark.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'Revenue']

In [28]:
# query 2. - Increasing count of total passengers in New York City by area


from pyspark.sql import functions as f
df_pyspark.groupBy('PULocationID').agg(f.sum('passenger_count').alias('total_passengers')) \
          .sort('total_passengers').show()

+------------+----------------+
|PULocationID|total_passengers|
+------------+----------------+
|         204|               1|
|          44|               1|
|         184|               2|
|         187|               2|
|           5|               3|
|          46|               3|
|         176|               3|
|         199|               4|
|          58|               4|
|           2|               4|
|          30|               4|
|         245|               5|
|         221|               6|
|         214|               6|
|         156|               8|
|         105|               8|
|         109|               8|
|          84|               8|
|         251|               9|
|          27|              12|
+------------+----------------+
only showing top 20 rows



In [29]:
# above query using sql 
spark.sql('select PULocationID,sum(passenger_count) as total_passenger from bda100_table group by PULocationID order by total_passenger').show(10)

+------------+---------------+
|PULocationID|total_passenger|
+------------+---------------+
|         204|              1|
|          44|              1|
|         184|              2|
|         187|              2|
|         176|              3|
|          46|              3|
|           5|              3|
|         199|              4|
|          58|              4|
|           2|              4|
+------------+---------------+
only showing top 10 rows



In [39]:
# Query 3. - Realtime Average fare/total earning amount earned by 2 vendors


from pyspark.sql import functions as f

# total number of vendors 


df_pyspark.select('VendorID').distinct().show()

total_fare_amount = df_pyspark.select('fare_amount').agg(f.sum('fare_amount').alias('total_fare_amount'))
total_amount = df_pyspark.select('total_amount').agg(f.sum('total_amount').alias('total_amount'))

+--------+
|VendorID|
+--------+
|       1|
|       2|
+--------+



In [56]:
total_fare_amount.show()
total_amount.show()
x = total_fare_amount.collect()
y = total_amount.collect()

# average of [fare / total]
print('Average ', x[0].total_fare_amount / y[0].total_amount)

+--------------------+
|   total_fare_amount|
+--------------------+
|1.0725890238000062E8|
+--------------------+

+------------------+
|      total_amount|
+------------------+
|1.35699048741755E8|
+------------------+

Average  0.7904174964713421


In [61]:
# Query 4. - Moving Count of payments made by each payment mode


# total payment_made by each payment mode
df_pyspark.groupBy('payment_type').agg(f.sum('Revenue').alias('total_amount')).show()


# total count of payment made by each payment mode 
df_pyspark.groupBy('payment_type').agg(f.count('Revenue').alias('tota_count')).show()

+------------+-------------------+
|payment_type|       total_amount|
+------------+-------------------+
|           1|2.046995811610078E8|
|           3| 1084041.3600001158|
|           4| 294450.68999999244|
|           2|6.528970845947792E7|
+------------+-------------------+

+------------+----------+
|payment_type|tota_count|
+------------+----------+
|           1|   6105871|
|           3|     43204|
|           4|     11852|
|           2|   2598947|
+------------+----------+



In [62]:
# above query usign spark sql 

df_pyspark.createOrReplaceTempView("bda100_table")
spark.sql('select payment_type,sum(Revenue) as total_revenue from bda100_table group by payment_type').show()

spark.sql('select payment_type,count(Revenue) as total_count from bda100_table group by payment_type').show()

+------------+-------------------+
|payment_type|      total_revenue|
+------------+-------------------+
|           1|2.046995811610078E8|
|           3| 1084041.3600001158|
|           4| 294450.68999999244|
|           2|6.528970845947792E7|
+------------+-------------------+

+------------+-----------+
|payment_type|total_count|
+------------+-----------+
|           1|    6105871|
|           3|      43204|
|           4|      11852|
|           2|    2598947|
+------------+-----------+



In [90]:
# Query 5. - Highest two gaining vendor's on a particular date with no of passenger and total distance by cab

df_pyspark.groupBy(f.to_date('tpep_pickup_datetime').alias('date')).agg(
                  f.sum('Revenue').alias('total_gain'),
                  f.sum('passenger_count').alias('total_passengers'),
                  f.sum('trip_distance').alias('total_distance')
).orderBy(f.sum('Revenue').desc()).show(2)

+----------+--------------------+----------------+-----------------+
|      date|          total_gain|total_passengers|   total_distance|
+----------+--------------------+----------------+-----------------+
|2018-01-25|1.0550837729971118E7|          523815|902081.6699999892|
|2018-01-26|1.0548607099968836E7|          536344|901778.8499999947|
+----------+--------------------+----------------+-----------------+
only showing top 2 rows



In [59]:
# Query 6. - Most no of passenger between a route of two location.

df_pyspark.groupBy('PULocationID','DOLocationID').agg(f.sum('passenger_count').alias('total_passengers')).show(10)

+------------+------------+----------------+
|PULocationID|DOLocationID|total_passengers|
+------------+------------+----------------+
|         114|         257|             171|
|          83|         142|               7|
|         229|         239|            3211|
|          25|          61|             401|
|         148|         229|            1995|
|         107|         161|           10588|
|         264|         107|             537|
|         151|         116|            1579|
|         231|          41|             380|
|         141|         173|             101|
+------------+------------+----------------+
only showing top 10 rows



In [60]:
# above query using sql
spark.sql('select PULocationID,DOLocationID,sum(passenger_count) as total_passengers \
          from bda100_table group by PULocationID,DOLocationID').show(10)

+------------+------------+----------------+
|PULocationID|DOLocationID|total_passengers|
+------------+------------+----------------+
|         114|         257|             171|
|          83|         142|               7|
|         229|         239|            3211|
|          25|          61|             401|
|         148|         229|            1995|
|         107|         161|           10588|
|         264|         107|             537|
|         151|         116|            1579|
|         231|          41|             380|
|         141|         173|             101|
+------------+------------+----------------+
only showing top 10 rows



In [ ]:
# Get top pickup locations with most passengers in last 5/10 seconds
# tpep_pickup_datetime + next 10 minutes 

